# 统一流形逼近投影(UMAP)

## 一、介绍
UMAP全称uniform manifold approximation and projection，统一流形逼近与投影，是继ISOMAP，LLE之后，流形学习在数据降维算法领域的又一力作。

UMAP和t-SNE降维算法除了用在数据预处理之外，还用在**可视化高维数据**上，UMAP有非常快的计算速度(比t-SNE更快)，可以实现大数据集的降维。

## 二、算法细节

UMAP对数据有三点假设：

* 数据均匀地分布在一个黎曼流形上。
* 黎曼度量是局部不变的。
* 黎曼流形是局部连接的。

通过这些假设，UMAP使用模糊拓扑结构表示高维数据。给定数据的一些低维表示，可以使用类似的过程来构造等价的拓扑表示。
UMAP在低维空间中优化数据坐标，以**最小化两个拓扑表示之间的交叉熵**，从而使得低维空间的表示可以同时保证局部和全局的结构。

### 2.1 构造带权无向图
UMAP首先将高维数据集$X = \{x_i\}_i$表示成一个带权无向图，具体来说，首先构造数据点的$k$-邻居图$\hat{G} = (V, E, W)$(带权有向图)，边的权重可以直观地理解为是该有向边存在的概率。

$$P_{i|j} = exp(-\frac{d(x_i, x_j)-\rho_i}{\sigma_i})$$

其中$\rho_i$表示第$i$个数据点与其最近邻之间的距离，$\sigma_i$是一个归一化系数。接着进行对称化操作，把权重变为如下形式，此时边变成无向边，权重可以直观地理解为至少有一边存在的概率。

$$P_{ij}=P_{i|j} + P_{j|i} - P_{i|j}P_{j|i}$$

### 2.2 构造低维表示并优化

构造了数据集$X$的带权无向图表示之后，再给出数据集低维空间(降维空间)图权重的定义方法，UMAP采取了类似$T$-分布的形式

$$q_{ij} = (1 +a(y_i - y_j)^{2b})^{-1}$$

UMAP使用二元交叉熵(CE)作为损失函数，去最小化高维空间中的带权无向图和低维空间中带权无向图对应边之间权重的差异。

$$CE(P, Q) = \sum_{i}\sum_{j}[P_{ij}log(\frac{P_{ij}}{Q_{ij}}) - (1 - P_{ij})log(\frac{1 - P_{ij}}{1 - Q_{ij}})]$$

UMAP使用SGD优化算法来优化二元交叉熵。

### 2.3 UMAP中的超参数
UMAP中有三个超参数
* 降维空间维度$d$。
* 构造$k$-邻居图的$k$。
* 最小允许距离$d_{min}$，用来防止降维空间中数据之间距离过近，从而导致点云稠密，降低可视化效果。

## 三、总结

ISOMAP也是流形学习在降维算法上的应用，ISOMAP也需要先将流形表示成一个$k$-邻居图，然后在该图上计算节点之间的图上最短路径，作为流形上测地距离的近似，构造距离矩阵，然后用MDS算法进行降维。ISOMAP是一种**保测地距离的降维算法**。ISOMAP算法较慢，计算图上任意两点之间距离需要花费较多时间。

相比之下，UMAP要高明地多，它放弃了保持任意两点之间测地距离这一全局结构，而是仅保持$k-$邻居图权重这一局部结构，但对降维可视化来说这已经足够了。

## 四、参考资料

[1] UMAP: Uniform Manifold Approximation and Projection for Dimension Reduction[J]. The Journal of Open Source Software, 2018, 3(29):861.